## Pretrained ResNet18 (Fixed)

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from tqdm import tqdm
from torchvision.models import resnet18
import numpy as np

In [2]:
args = {
    'learning_rate': 5e-2,
    'batch_size': 16,
    'num_worker': 4,
    'random_seed': 8771795,
    'augmentation': False,
    'num_epoch': 20,
    'cuda': '3'
}

In [3]:
# set device
os.environ["CUDA_VISIBLE_DEVICES"] = args['cuda']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed
torch.random.manual_seed(args['random_seed'])

# Define transformation
test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_valid_transform = test_transform
if args['augmentation']:
    train_valid_transform = transforms.Compose([
    #         transforms.RandomResizedCrop((224, 224)),
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.RandomErasing(),
            transforms.Normalize((0.5,), (0.5,))
        ])

# Load dataset
require_download = os.path.exists('./dataset')
train_valid_dataset = torchvision.datasets.FashionMNIST('./dataset', train=True, transform=train_valid_transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST('./dataset', train=False, transform=test_transform, download=True)

# Split train and validation
torch.random.manual_seed(args['random_seed'])
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [54000, 6000])

# Generate dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_worker'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])

In [4]:
model = resnet18(pretrained=True)

In [5]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [6]:
model.fc = Identity()

In [7]:
model.eval()
print('total parameters:', sum(p.numel() for p in model.parameters()))

total parameters: 11176512


In [8]:
if not os.path.exists('./features'):
    os.mkdir('./features')

In [9]:
model = model.to(device)

In [10]:
train_feats, valid_feats, test_feats = [], [], []
train_labels, valid_labels, test_labels = [], [], []
for iteration, (x, y) in tqdm(enumerate(train_loader)):
    x, y = x.to(device), y.to(device)
    features = model(x)
    train_feats.append(features.cpu().detach().numpy())
    train_labels.append(y.cpu().detach().numpy())
    del x, y, features

3375it [00:43, 76.95it/s]


In [11]:
train_feats = np.concatenate(train_feats)

In [12]:
train_feats.shape

(54000, 512)

In [13]:
train_labels = np.concatenate(train_labels)
train_labels.shape

(54000,)

In [14]:
np.save('features/resnet18_train_feat.npy', train_feats)
np.save('features/resnet18_train_label.npy', train_labels)

In [15]:
del train_labels, train_feats

In [16]:
for iteration, (x, y) in tqdm(enumerate(valid_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    features = model(x)
    valid_feats.append(features.cpu().detach().numpy())
    valid_labels.append(y.cpu().detach().numpy())
    del x, y, features

375it [00:04, 76.21it/s]


In [17]:
valid_feats = np.concatenate(valid_feats)
valid_feats.shape

(6000, 512)

In [18]:
valid_labels = np.concatenate(valid_labels)
valid_labels.shape

(6000,)

In [19]:
np.save('features/resnet18_valid_feat.npy', valid_feats)
np.save('features/resnet18_valid_label.npy', valid_labels)

In [20]:
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    features = model(x)
    test_feats.append(features.cpu().detach().numpy())
    test_labels.append(y.cpu().detach().numpy())
    del x, y, features

625it [00:08, 75.55it/s]


In [21]:
test_feats = np.concatenate(test_feats)
test_feats.shape

(10000, 512)

In [22]:
test_labels = np.concatenate(test_labels)
test_labels.shape

(10000,)

In [23]:
np.save('features/resnet18_test_feat.npy', test_feats)
np.save('features/resnet18_test_label.npy', test_labels)

## Pretrained ResNet18 (Finetune)

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from tqdm import tqdm
from torchvision.models import resnet18
import numpy as np

In [3]:
args = {
    'learning_rate': 1e-3,
    'batch_size': 32,
    'num_worker': 4,
    'random_seed': 8771795,
    'augmentation': False,
    'num_epoch': 10,
    'cuda': '3'
}

In [4]:
# set device
os.environ["CUDA_VISIBLE_DEVICES"] = args['cuda']
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Set random seed
torch.random.manual_seed(args['random_seed'])

# Define transformation
test_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

train_valid_transform = test_transform
if args['augmentation']:
    train_valid_transform = transforms.Compose([
    #         transforms.RandomResizedCrop((224, 224)),
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
            transforms.RandomErasing(),
            transforms.Normalize((0.5,), (0.5,))
        ])

# Load dataset
require_download = os.path.exists('./dataset')
train_valid_dataset = torchvision.datasets.FashionMNIST('./dataset', train=True, transform=train_valid_transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST('./dataset', train=False, transform=test_transform, download=True)

# Split train and validation
torch.random.manual_seed(args['random_seed'])
train_dataset, valid_dataset = torch.utils.data.random_split(train_valid_dataset, [54000, 6000])

# Generate dataloader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=args['num_worker'])
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=args['batch_size'], shuffle=False, num_workers=args['num_worker'])

### Train only the last FC layer

In [4]:
model = resnet18(pretrained=True)
for param in model.parameters():
    param.require_grad = False
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [5]:
model.fc = nn.Linear(in_features=512, out_features=10, bias=True)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=args['learning_rate'])

In [7]:
model.to(device)
for epoch in range(args['num_epoch']):
    total_train = 0
    for iteration, (x, y) in tqdm(enumerate(train_loader)):
        model.train()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)
        total_train += loss.item()
        loss.backward()
        optimizer.step()
        del x, y, pred
    print('epoch {}: training loss: {}'.format(epoch+1, total_train/(iteration+1)))
    torch.cuda.empty_cache()

    val_acc = 0
    for iteration, (x, y) in tqdm(enumerate(valid_loader)):
        model.eval()
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = torch.argmax(out, dim=1)
        acc = sum(pred == y)
        val_acc += acc.item()
    print('epoch {}: valid acc: {}'.format(epoch+1, val_acc/6000))

1688it [01:45, 16.04it/s]

epoch 1: training loss: 1.6946770741087849



188it [00:04, 41.77it/s]

epoch 1: valid acc: 0.5913333333333334



1688it [01:45, 15.96it/s]

...

epoch 9: valid acc: 0.845



1688it [01:47, 15.75it/s]

epoch 10: training loss: 0.441060103965985



188it [00:04, 40.12it/s]

epoch 10: valid acc: 0.8316666666666667


In [8]:
test_acc = 0
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
print('test acc: {}'.format(test_acc/10000))

313it [00:07, 43.65it/s]

test acc: 0.8273


In [9]:
torch.save(model.state_dict(), 'resnet18_before.pt')

### Fine-tune all the layers

In [16]:
model = resnet18()
model.fc = nn.Linear(512, 10)
model.load_state_dict(torch.load('resnet18_before.pt'))

<All keys matched successfully>

In [17]:
for param in model.parameters():
    param.require_grad = True

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)

In [19]:
model.to(device)
for epoch in range(10):
    total_train = 0
    for iteration, (x, y) in tqdm(enumerate(train_loader)):
        model.train()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = criterion(pred, y)
        total_train += loss.item()
        loss.backward()
        optimizer.step()
        del x, y, pred
    print('epoch {}: training loss: {}'.format(epoch+1, total_train/(iteration+1)))
    torch.cuda.empty_cache()

    val_acc = 0
    for iteration, (x, y) in tqdm(enumerate(valid_loader)):
        model.eval()
        x, y = x.to(device), y.to(device)
        out = model(x)
        pred = torch.argmax(out, dim=1)
        acc = sum(pred == y)
        val_acc += acc.item()
    print('epoch {}: valid acc: {}'.format(epoch+1, val_acc/6000))

1688it [01:44, 16.12it/s]

epoch 1: training loss: 0.4344776571436975



188it [00:04, 40.42it/s]

epoch 1: valid acc: 0.8476666666666667



1688it [01:45, 15.94it/s]

...

epoch 4: valid acc: 0.8575



1688it [01:46, 15.85it/s]

epoch 5: training loss: 0.40058715476420537



188it [00:04, 39.58it/s]

epoch 5: valid acc: 0.8591666666666666


In [9]:
# lr = 1e-5 * 10
test_acc = 0
for iteration, (x, y) in tqdm(enumerate(test_loader)):
    model.eval()
    x, y = x.to(device), y.to(device)
    out = model(x)
    pred = torch.argmax(out, dim=1)
    acc = sum(pred == y)
    test_acc += acc.item()
print('test acc: {}'.format(test_acc/10000))

313it [00:07, 41.84it/s]

test acc: 0.8554


In [10]:
torch.save(model.state_dict(), 'resnet18_after.pt')